#### Available scene list

In [17]:
import json

    
f = open('../scene.json', "r")
scene_json = json.load(f)

scene_list = list(scene_json.keys())
scene_dict = scene_json

#### Generate scene csv

In [30]:
import os
import pandas as pd


def generate_csv(s_idx, r_idx, sp_nums=None):
    
    scene_id = scene_list[s_idx]
    
    # Data (image and ground truth) directory
    ds_pth = f"E:/Workspace/Datasets/iGibson-dataset/iGibson-pano-data/"
    ds_pth = f"{ds_pth}/{scene_id}/{r_idx}/"
    if sp_nums is None:
        sp_nums = len(os.listdir(f"{ds_pth}/"))
    
    # Map (wall and object) directory
    mp_pth = f"E:/Workspace/Datasets/iGibson-dataset/iGibson-area/"
    mp_pth = f"{mp_pth}/{scene_id}/{r_idx}/wmap.txt"
    
    # Store data directory in csv
    tgt_pth = f"E:/Workspace/Datasets/iGibson-dataset/iGibson-pano-csv/"
    if not os.path.exists(f"{tgt_pth}/{scene_id}/"):
        os.makedirs(f"{tgt_pth}/{scene_id}/")
    tgt_pth = f"{tgt_pth}/{scene_id}/{r_idx}.csv"
    
    scene_df = {
        "gt": [],
        "rgb": [],
        "d": [],
        "mp": []
    }
    
    for i in range(sp_nums):
        
        gt_pth = f"{ds_pth}/{i}/gt/"
        scene_df["gt"].append(gt_pth)
        
        rgb_pth = f"{ds_pth}/{i}/rgb/"
        scene_df["rgb"].append(rgb_pth)
        
        # d_pth = f"{ds_pth}/{i}/d_processed/"
        d_pth = f"{ds_pth}/{i}/d/"
        scene_df["d"].append(d_pth)
        
        scene_df["mp"].append(mp_pth)
        
    scene_df = pd.DataFrame(scene_df)
    scene_df.to_csv(tgt_pth, index=False)

In [31]:
s_idx = 1
for r_idx in range(5):
    generate_csv(s_idx, r_idx)

s_idx = 3
for r_idx in range(2):
    generate_csv(s_idx, r_idx)
    
s_idx = 4
for r_idx in range(6):
    generate_csv(s_idx, r_idx)
    
s_idx = 5
for r_idx in range(3):
    generate_csv(s_idx, r_idx)
    
s_idx = 6
for r_idx in range(6):
    generate_csv(s_idx, r_idx)

#### Train and test split

In [32]:
import json

    
f = open('../train.json', "r")
scene_json = json.load(f)

train_list = scene_json["train_list"]
test_list = scene_json["test_list"]

In [33]:
import pandas as pd


ds_pth = f"E:/Workspace/Datasets/iGibson-dataset/iGibson-pano-csv"

train_df = pd.DataFrame()
for idx in range(len(train_list)):
    
    train_item = pd.read_csv(
        f"{ds_pth}/{train_list[idx]}"
    )
    train_df = pd.concat((train_df, train_item))

train_df.to_csv(
    f"{ds_pth}/train.csv",
    index=False
)

test_df = pd.DataFrame()
for idx in range(len(test_list)):
    
    test_item = pd.read_csv(
        f"{ds_pth}/{test_list[idx]}"
    )
    test_df = pd.concat((test_df, test_item))

test_df.to_csv(
    f"{ds_pth}/test.csv",
    index=False
)

#### Validate csv file

In [34]:
import pandas as pd


ds_pth = f"E:/Workspace/Datasets/iGibson-dataset/iGibson-pano-csv"

train_pth = f"{ds_pth}/train.csv"
train_df = pd.read_csv(train_pth)

train_sample = train_df.sample(n=5)
print(train_sample.index)
print(train_sample.columns)

print(train_sample["gt"][train_sample.index[0]])
print(train_sample["rgb"][train_sample.index[1]])
print(train_sample["d"][train_sample.index[2]])
print(train_sample["mp"][train_sample.index[3]])

test_pth = f"{ds_pth}/test.csv"
test_df = pd.read_csv(test_pth)

test_sample = test_df.sample(n=5)
print(test_sample.index)
print(test_sample.columns)

print(test_sample["gt"][test_sample.index[0]])
print(test_sample["rgb"][test_sample.index[1]])
print(test_sample["d"][test_sample.index[2]])
print(test_sample["mp"][test_sample.index[3]])

Index([424, 417, 505, 488, 278], dtype='int64')
Index(['gt', 'rgb', 'd', 'mp'], dtype='object')
E:/Workspace/Datasets/iGibson-dataset/iGibson-pano-data//Ihlen_1_int/2//24/gt/
E:/Workspace/Datasets/iGibson-dataset/iGibson-pano-data//Ihlen_1_int/2//17/rgb/
E:/Workspace/Datasets/iGibson-dataset/iGibson-pano-data//Merom_1_int/0//5/d/
E:/Workspace/Datasets/iGibson-dataset/iGibson-area//Merom_0_int/0/wmap.txt
Index([20, 18, 83, 108, 139], dtype='int64')
Index(['gt', 'rgb', 'd', 'mp'], dtype='object')
E:/Workspace/Datasets/iGibson-dataset/iGibson-pano-data//Beechwood_1_int/4//20/gt/
E:/Workspace/Datasets/iGibson-dataset/iGibson-pano-data//Beechwood_1_int/4//18/rgb/
E:/Workspace/Datasets/iGibson-dataset/iGibson-pano-data//Merom_0_int/1//33/d/
E:/Workspace/Datasets/iGibson-dataset/iGibson-area//Merom_1_int/1/wmap.txt


#### Panorama dataset

In [40]:
import pandas as pd

from torch.utils.data import Dataset
from utils import *


class IgibsonPanorama(Dataset):
    
    def __init__(self, csv_pth):
        super(IgibsonPanorama, self).__init__()
        
        self.data = pd.read_csv(csv_pth)
        
    def __len__(self):
        
        return len(self.data)
    
    def __getitem__(self, index):
        
        # Read ground truth
        gt_pth = self.data["gt"][index]
        gt = read_gt(gt_pth)
        gt = gt[0, ...]
        
        # Read observation
        # obs_pth = self.data["rgb"][index]
        # obs = read_obs(obs_pth)
        
        obs_pth = self.data["d"][index]
        obs = read_depth(obs_pth)
        
        # Read map
        mp_pth = self.data["mp"][index]
        mp, mp_mask = read_mp(mp_pth)
        
        return (gt, obs, mp, mp_mask)

In [42]:
from tqdm import tqdm

from torch.utils.data import DataLoader


train_pth = "e:\Workspace\Datasets\iGibson-dataset\iGibson-pano-csv/train.csv"
train_dataset = IgibsonPanorama(train_pth)
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)

gt, obs, mp, mp_mask = next(iter(train_dataloader))
print(f"Ground truth: {gt.shape}")
print(f"Observation {obs.shape}")
print(f"Map {mp.shape}")
print(f"Map mask {mp_mask.shape}")
print(f"Map mask {mp_mask}")

for gt, obs, mp, mp_mask in tqdm(train_dataloader):
    pass

test_pth = "e:\Workspace\Datasets\iGibson-dataset\iGibson-pano-csv/test.csv"
test_dataset = IgibsonPanorama(test_pth)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=True)

gt, obs, mp, rel = next(iter(test_dataloader))
print(f"Ground truth: {gt.shape}")
print(f"Observation {obs.shape}")
print(f"Map {mp.shape}")
print(f"Map mask {mp_mask.shape}")
print(f"Map mask {mp_mask}")

for gt, obs, mp, rel in tqdm(test_dataloader):
    pass


Ground truth: torch.Size([4, 3])
Observation torch.Size([4, 1, 8, 3, 224, 224])
Map torch.Size([4, 20, 4])
Map mask torch.Size([4, 20])
Map mask tensor([[1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.]])


100%|██████████| 150/150 [00:23<00:00,  6.40it/s]


Ground truth: torch.Size([4, 3])
Observation torch.Size([4, 1, 8, 3, 224, 224])
Map torch.Size([4, 20, 4])
Map mask torch.Size([4, 20])
Map mask tensor([[1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.]])


100%|██████████| 50/50 [00:07<00:00,  6.68it/s]
